In [1]:
import pandas as pd
import numpy as np
import re
import xlrd
import pickle
import os
import requests
# from bs4 import BeautifulSoup
# or import bs4 as bs
# import json

In [2]:
# setting directories for file loads and saves
logs_dir = "../data/logs/"
raw_dir = "../data/raw/"
load_dir = save_dir = "../data/interim/"
final_dir = "../data/processed/"

In [6]:
xl = pd.read_pickle(load_dir + "school_profiles_2008_2017_df.pickle")
xl.head()

calendar_year  school_id                     school_name     suburb state  \
0           2008      40000  Corpus Christi Catholic School  BELLERIVE   TAS   
1           2009      40000  Corpus Christi Catholic School  BELLERIVE   TAS   
2           2010      40000  Corpus Christi Catholic School  BELLERIVE   TAS   
3           2011      40000  Corpus Christi Catholic School  BELLERIVE   TAS   
4           2012      40000  Corpus Christi Catholic School  BELLERIVE   TAS   

   postcode school_sector school_type   icsea  bottom_sea_quarter  \
0      7018      Catholic     Primary  1055.0                13.0   
1      7018      Catholic     Primary  1055.0                13.0   
2      7018      Catholic     Primary  1044.0                28.0   
3      7018      Catholic     Primary  1046.0                19.0   
4      7018      Catholic     Primary  1041.0                 9.0   

   lower_middle_sea_quarter  upper_middle_sea_quarter  top_sea_quarter  \
0                      19.0                      37.0             31.0   
1                      19.0                      37.0             31.0   
2                      19.0                      30.0             23.0   
3                      30.0                      25.0             25.0   
4                      35.0                      28.0             28.0   

   fte_teachers  fte_other_staff  total_enrolments  girls_enrolments  \
0          19.0              6.9             410.0             185.0   
1          19.1              8.7             409.0             184.0   
2          18.6             10.2             392.0             173.0   
3          18.1              9.0             385.0             173.0   
4          18.1              9.4             391.0             185.0   

   boys_enrolments  indigenous_enrolments  lote  
0            225.0                    1.0   NaN  
1            225.0                    2.0   3.0  
2            219.0                    2.0   4.0  
3            212.0                    0.0   2.0  
4            206.0                    1.0   5.0

In [4]:
xl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93080 entries, 0 to 93079
Data columns (total 4 columns):
Calendar Year      93080 non-null int64
ACARA School ID    93080 non-null int64
School Type        93080 non-null object
State              93080 non-null object
dtypes: int64(2), object(2)
memory usage: 2.8+ MB


# MySchool website, Sherwood State School

In [5]:
url = "https://www.myschool.edu.au/school/46461/naplan/similar/2017"

# Packages the request, send the request and catch the response: r
r = requests.get(url)

# Extract the response: text
text = r.text

# Print the html
text[:500]

'\r\n<!doctype html>\r\n<html class="no-js" lang="en-AU">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\r\n    <title>Similar schools | My School</title>\r\n    <meta name="description" content="">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no">\r\n    <link rel="icon" href="/favicon.ico" type="image/x-icon">\r\n    <link rel="apple-touch-icon" href="apple-touch-icon.png">\r\n    <!-- Styles -->\r\n    <link rel="stylesh'

In [6]:
# using BeautifulSoup, turn html into cleaner text
soup = BeautifulSoup(text, "lxml")


In [7]:
print(soup.prettify()[:500])

<!DOCTYPE html>
<html class="no-js" lang="en-AU">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <title>
   Similar schools | My School
  </title>
  <meta content="" name="description"/>
  <meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
  <link href="/favicon.ico" rel="icon" type="image/x-icon"/>
  <link href="apple-touch-icon.png" rel="apple-touch-icon"/>
  <!-- Styles -->
  <link href="https://cdnjs.cloudflar


In [8]:
# Find all 'script' tags (which define scripts): scripts
scripts = soup.find_all('script')

# Print the school data <script> to the shell
#print(scripts[6])

# Save the school data <script> to a text file


In [9]:
# adding data for POST to retrieve different SchoolYearId, DomainId and ViewModeId
payload = {"SchoolYearId" : "5", "DomainId" : "1", "ViewModeId" : "0"}

# retrieving the above from the same page as r
r510 = requests.post(url, data = payload)

# Extract the response: text
text510 = r510.text

# using BeautifulSoup, turn html into cleaner text
soup510 = BeautifulSoup(text510, "lxml")

In [10]:
# Find all 'script' tags (which define scripts): scripts
scripts510 = soup510.find_all('script')

# Print the head of scipts to the shell
#print(scripts510)

In [11]:
type(scripts[6])

bs4.element.Tag

In [23]:
type(scripts[6].contents[0].string)

bs4.element.NavigableString

In [27]:
# This gives a string which is writable to file
scripts[6].string[:50]

'if (document.addEventListener) {document.addEventL'

In [26]:
# Writing to files
f = open("sherwood310.txt", mode = "w", encoding = "utf-8")
f.write(scripts[6].string)
f.close()

f = open("sherwood510.txt", mode = "w", encoding = "utf-8")
f.write(scripts510[6].string)
f.close()

In [16]:
scripts[6].attrs

{'type': 'text/javascript'}

In [29]:
s310 = scripts[6].string
type(s310)

bs4.element.NavigableString

In [30]:
s310slash =  re.sub(r'\\', '', s310)
s310colq = re.sub(r':"', ':', s310slash)
s310qcomma = re.sub(r'",', ',', s310colq)

In [33]:
s310qcomma[:1000]

'if (document.addEventListener) {document.addEventListener("DOMContentLoaded, function() {createChartnaplanSimilarSchoolsBubble();});} else if (document.attachEvent) {document.attachEvent("onreadystatechange, function(){if (document.readyState === "complete"){document.detachEvent("onreadystatechange, arguments.callee);createChartnaplanSimilarSchoolsBubble();}});}function createChartnaplanSimilarSchoolsBubble() {var ChartOptions = {"credits":{"enabled":false},"exporting":{"enabled":false},"xAxis":[{"endOnTick":true,"min":440,"tickInterval":5,"max":540,"tickLength":0,"title":{"text":Average achievement score,"offset":50},"labels":{"step":2}}],"chart":{"height":408,"style":{"min-width":390px"},"renderTo":naplanSimilarSchoolsBubble"},"yAxis":[{"tickInterval":1,"min":0,"labels":{"enabled":false},"max":12,"title":{"text":Number of schools"},"lineWidth":1}],"tooltip":{"style":{"padding":0,"pointerEvents":auto"},"hideDelay":1000,"formatter":tooltipbuilder,"useHTML":true},"title":{"text":"},"se

In [35]:
s310up2data = re.sub(r'.*?data":', '', s310qcomma)
s310up2data[:500]

'[{"x":450,"description":{"schoolId":49793,"mean":449,"lowerMargin":431,"upperMargin":467,"schoolName":Concordia College"},"marker":{"symbol":url(/Content/images/naplan/graphs/bubble-4.png)"},"y":0.5},{"x":450,"description":{"schoolId":45869,"mean":451,"lowerMargin":433,"upperMargin":469,"schoolName":St Bedeu0027s School"},"marker":{"symbol":url(/Content/images/naplan/graphs/bubble-4.png)"},"y":1.5},{"x":455,"description":{"schoolId":41234,"mean":454,"lowerMargin":441,"upperMargin":467,"schoolNam'

In [74]:
s310apostro = re.sub(',"plotOptions(.*)', '', s310up2data)
s310datalist = re.sub("u0027", "'", s310apostro)
print(s310up2data[-500:])
s310datalist[-500:]

00,"upperMargin":551,"schoolName":Toowong State School"},"marker":{"symbol":url(/Content/images/naplan/graphs/bubble-1.png)"},"y":1.5},{"x":530,"description":{"schoolId":44855,"mean":528,"lowerMargin":512,"upperMargin":545,"schoolName":Beverley Hills Primary School"},"marker":{"symbol":url(/Content/images/naplan/graphs/bubble-1.png)"},"y":0.5}]}],"plotOptions":{"series":{"states":{"hover":{"enabled":false}},"marker":{"radius":0}}}};new Highcharts.chart("naplanSimilarSchoolsBubble,ChartOptions);}


'hool"},"marker":{"symbol":url(/Content/images/naplan/graphs/bubble-1.png)"},"y":0.5},{"x":525,"description":{"schoolId":46402,"mean":525,"lowerMargin":500,"upperMargin":551,"schoolName":Toowong State School"},"marker":{"symbol":url(/Content/images/naplan/graphs/bubble-1.png)"},"y":1.5},{"x":530,"description":{"schoolId":44855,"mean":528,"lowerMargin":512,"upperMargin":545,"schoolName":Beverley Hills Primary School"},"marker":{"symbol":url(/Content/images/naplan/graphs/bubble-1.png)"},"y":0.5}]}]'

In [ ]:
f = open('s310datalist.txt', mode = 'w', encoding = 'utf-8')
f.write(s310datalist)

In [79]:
grade = 3
domain = 1
view = 0

year = 2017

In [80]:
schoolIdList = re.findall('schoolId":(\d\d\d\d\d)', s310datalist)
meanList = re.findall('mean":(\d\d\d)', s310datalist)
lowerList = re.findall('lowerMargin":(\d\d\d)', s310datalist)
upperList = re.findall('upperMargin":(\d\d\d)', s310datalist)

sherwood_310_df = pd.DataFrame({'schoolId' : schoolIdList, 'grade' : grade, 
                                'year' : year, 'mean' : meanList, 'lower' : lowerList,
                               'upper' : upperList})

sherwood_310_df.head()

schoolId  grade  year mean lower upper
0    49793      3  2017  449   431   467
1    45869      3  2017  451   433   469
2    41234      3  2017  454   441   467
3    47690      3  2017  454   439   470
4    46463      3  2017  456   434   477

In [ ]:
sherwood_310_df.to_csv('sherwood_310_df.csv')

# Generic school, from the all school IDs list

In [81]:
# adding data for POST to retrieve similar school list (ViewModeId = 1)
payload = {"SchoolYearId" : "5", "DomainId" : "1", "ViewModeId" : "1"}

# retrieving the above from the same page as r
r511 = requests.post(url, data = payload)

# Extract the response: text
text511 = r511.text

# using BeautifulSoup, turn html into cleaner text
soup511 = BeautifulSoup(text510, "lxml")

In [84]:
f = open('soup511_pretty.html', mode = 'w', encoding = 'utf-8')
f.write(soup511.prettify())
f.close()

In [103]:
scripts511 = soup511.find_all('script')
s311schoolList = scripts511[6].string

In [113]:
scripts511[6].string[-500:]

'\\"schoolName\\":\\"Leichhardt Public School\\"}","marker":{"symbol":"url(/Content/images/naplan/graphs/bubble-2.png)"},"y":0.5},{"x":595,"description":"{\\"schoolId\\":50402,\\"mean\\":593,\\"lowerMargin\\":574,\\"upperMargin\\":612,\\"schoolName\\":\\"Geelong Grammar School\\"}","marker":{"symbol":"url(/Content/images/naplan/graphs/bubble-2.png)"},"y":0.5}]}],"plotOptions":{"series":{"states":{"hover":{"enabled":false}},"marker":{"radius":0}}}};new Highcharts.chart("naplanSimilarSchoolsBubble",ChartOptions);}'

In [117]:
simimlar_schools_2017 = re.findall(r'schoolId\\":(\d\d\d\d\d)', scripts511[6].string)
#simimlar_schools_2017

## School profiles 2008 - 2017

In [50]:
file = load_dir + "school-profile-2008-2017.xlsx"
xl = pd.read_excel(file, sheet_name="School Profile")
print(xl.dtypes)
xl.head()


Calendar Year                                int64
ACARA School ID                              int64
AGE ID                                     float64
School Name                                 object
Suburb                                      object
State                                       object
Postcode                                     int64
School Sector                               object
School Type                                 object
Campus Type                                 object
Rolled Reporting Description                object
School URL                                  object
Year Range                                  object
Geolocation                                 object
ICSEA                                      float64
Bottom SEA Quarter                         float64
Lower Middle SEA Quarter                   float64
Upper Middle SEA Quarter                   float64
Top SEA Quarter                            float64
Teaching Staff                 

Calendar Year  ACARA School ID  AGE ID                     School Name  \
0           2008            40000     3.0  Corpus Christi Catholic School   
1           2009            40000     3.0  Corpus Christi Catholic School   
2           2010            40000     3.0  Corpus Christi Catholic School   
3           2011            40000     3.0  Corpus Christi Catholic School   
4           2012            40000     3.0  Corpus Christi Catholic School   

      Suburb State  Postcode School Sector School Type           Campus Type  \
0  BELLERIVE   TAS      7018      Catholic     Primary  School Single Entity   
1  BELLERIVE   TAS      7018      Catholic     Primary  School Single Entity   
2  BELLERIVE   TAS      7018      Catholic     Primary  School Single Entity   
3  BELLERIVE   TAS      7018      Catholic     Primary  School Single Entity   
4  BELLERIVE   TAS      7018      Catholic     Primary  School Single Entity   

                    ...                   Teaching Staff  \
0                   ...                             29.0   
1                   ...                             27.0   
2                   ...                             26.0   
3                   ...                             25.0   
4                   ...                             24.0   

  Full Time Equivalent Teaching Staff Non-Teaching Staff  \
0                                19.0               12.0   
1                                19.1               15.0   
2                                18.6               18.0   
3                                18.1               16.0   
4                                18.1               16.0   

  Full Time Equivalent Non-Teaching Staff  Total Enrolments  Girls Enrolments  \
0                                     6.9             410.0             185.0   
1                                     8.7             409.0             184.0   
2                                    10.2             392.0             173.0   
3                                     9.0             385.0             173.0   
4                                     9.4             391.0             185.0   

   Boys Enrolments  Full Time Equivalent Enrolments  Indigenous Enrolments  \
0            225.0                            410.0                    1.0   
1            225.0                            409.0                    2.0   
2            219.0                            392.0                    2.0   
3            212.0                            385.0                    0.0   
4            206.0                            391.0                    1.0   

   Language Background Other Than English  
0                                     NaN  
1                                     3.0  
2                                     4.0  
3                                     2.0  
4                                     5.0  

[5 rows x 29 columns]

In [11]:
allSchoolIdsList = xl['ACARA School ID'].unique().tolist();

In [ ]:
pickle_on = open("allSchoolIdsList.pickle", "wb")
pickle.dump(allSchoolIdsList, pickle_on)
pickle_on.close()

In [14]:
xl.to_pickle("allSchoolProfiles_df.pickle")

In [16]:
years = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']

In [ ]:
years_int = [int(y) for y in years]

In [51]:
#xl_dtypes_dict = xl.dtypes.to_dict()
#xl_dtypes_dict

In [9]:
all_schools_sort_list = ['Calendar Year', 'School Type', 'ACARA School ID']
all_school_years_df = xl.iloc[:, [0,1,8,5]].sort_values(all_schools_sort_list
                                                ).set_index('Calendar Year')

all_school_years_df.head()

ACARA School ID School Type State
Calendar Year                                   
2008                     40001    Combined   TAS
2008                     40002    Combined   TAS
2008                     40005    Combined   TAS
2008                     40007    Combined   TAS
2008                     40008    Combined   TAS

In [12]:
all_school_years_df.to_pickle(load_dir + "all_school_years_df.pickle")

In [53]:
year_df = all_school_years_df.loc[years_int[0]]
one_yr_list = year_df['ACARA School ID'].values.tolist()
one_yr_list[:5]

[40001, 40002, 40005, 40007, 40008]

In [59]:
year_df = all_school_years_df.loc[(years_int[-1]) & (
    all_school_years_df['School Type'] == 'Primary')]
primary_2017_schoolID_list = year_df['ACARA School ID'].values.tolist()
primary_2017_schoolID_list[:5]

[40000, 40003, 40004, 40006, 40009]

In [77]:
xl.columns

Index(['Calendar Year', 'ACARA School ID', 'AGE ID', 'School Name', 'Suburb',
       'State', 'Postcode', 'School Sector', 'School Type', 'Campus Type',
       'Rolled Reporting Description', 'School URL', 'Year Range',
       'Geolocation', 'ICSEA', 'Bottom SEA Quarter',
       'Lower Middle SEA Quarter', 'Upper Middle SEA Quarter',
       'Top SEA Quarter', 'Teaching Staff',
       'Full Time Equivalent Teaching Staff', 'Non-Teaching Staff',
       'Full Time Equivalent Non-Teaching Staff', 'Total Enrolments',
       'Girls Enrolments', 'Boys Enrolments',
       'Full Time Equivalent Enrolments', 'Indigenous Enrolments',
       'Language Background Other Than English'],
      dtype='object')